<a href="https://colab.research.google.com/github/david-levin11/Verification_Notebooks/blob/main/PlotHiRes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Objective: To plot archived high resolution model data for AK**
<br/>
Description--This script will attempt to use the Herbie python package to download and plot archived hi-resolution model data for case reviews/operational recaps.

The available models are:

**AK HRRR**<br/>
**RRFS Control Run**<br/>
**NamNest**

HRRR data for Alaska is available on AWS back to 2018.  NamNest data goes back to September 2021. RRFS data is sporadic as it is an experimental model but most of the data from 2024 is available on AWS.  Sometimes runs will not be found or be incomplete.  This may cause issues especially with precipitation variables which rely on two timesteps to be available in order to plot properly.

I have the notebook set to plot only a few of the numerous variables the hi-res models output.  If there is a particular variable you would like to see plotted, feel free to let me know and I'll see if I can add it.  Alternatively you are welcome to make a copy of this notebook and play around with adding various improvements!

**Update: 9/12/2024**
--Added custom zoom areas as well as the ability to create animated gifs.  Let me know if you'd like different zooms or any other features/fields!

- David Levin, Arctic Testbed & Proving Ground, Anchorage Alaska

##**2 - Install and Import Packages**
This will take about a minute to run.

In [ ]:
# @title
!pip install ecmwflibs
!pip install eccodes
!pip install cfgrib
!pip install curl
!pip install eccodes
!pip install wgrib2
!pip install herbie-data[extras]
!pip install ipywidgets
from herbie import Herbie
from herbie.toolbox import EasyMap, pc
from herbie import paint
try:
    import numpy as np
    import os
    from datetime import datetime, timedelta
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
except ImportError:
    raise ImportError("herbie.paint requires matplotlib.")
from PIL import Image

##**3 - Download & Plot**

In [ ]:
###################### Classes for Colormaps ##################################
# setting up our colorscales for plotting precip since Herbie only uses mm    #
# can use paint for everything else                                           #
###############################################################################
def make_custom_cmaps(name, colors, bounds: list = None, N: int = None):
    if N is None:
        N = len(colors)
    linear_cmap = mcolors.LinearSegmentedColormap.from_list(name, colors)
    segment_cmap = mcolors.LinearSegmentedColormap.from_list(name + "2", colors, N=N)

    # When data is NaN, set color to transparent
    linear_cmap.set_bad("#ffffff00")
    segment_cmap.set_bad("#ffffff00")

    for cm in [linear_cmap, segment_cmap]:
        mpl.colormaps.register(cmap=cm, force=True)
        mpl.colormaps.register(cmap=cm.reversed(), force=True)

    if bounds is not None:
        return (
            mcolors.Normalize(bounds.min(), bounds.max()),
            mcolors.BoundaryNorm(bounds, linear_cmap.N),
        )

class NWSPrecipitation:
    """National Weather Service precipitation amount colorbar properties.

    Also known as Qualitative Precipitation Forecast/Estimate (QPF/QPE).
    """

    name = "nws.pcp"
    units = "in"
    variable = "Precipitation"
    colors = np.array(
        [
            "#ffffff",
            "#c7e9c0",
            "#a1d99b",
            "#74c476",
            "#31a353",
            "#006d2c",
            "#fffa8a",
            "#ffcc4f",
            "#fe8d3c",
            "#fc4e2a",
            "#d61a1c",
            "#ad0026",
            "#700026",
            "#3b0030",
            "#4c0073",
            "#ffdbff",
        ]
    )
    # NWS bounds in inches
    bounds = np.array(
        [0, 0.01, 0.1, 0.25, 0.5, 1, 1.5, 2, 3, 4, 6, 8, 10, 15, 20, 30, 50]
    )
    norm, norm2 = make_custom_cmaps(name, colors, bounds)
    cmap = plt.get_cmap(name)
    cmap2 = plt.get_cmap(name + "2")
    kwargs = dict(cmap=cmap, norm=norm)
    kwargs2 = dict(cmap=cmap, norm=norm2)
    cbar_kwargs = dict(label=f"{variable} ({units})")
    cbar_kwargs2 = cbar_kwargs | dict(spacing="uniform", ticks=bounds)

class NWSWindSpeed:
    name = "nws.wind"
    units = r"mph"
    variable = "Wind Speed"
    colors = np.array(
        [
            "#103f78",
            "#225ea8",
            "#1d91c0",
            "#41b6c4",
            "#7fcdbb",
            "#b4d79e",
            "#dfff9e",
            "#ffffa6",
            "#ffe873",
            "#ffc400",
            "#ffaa00",
            "#ff5900",
            "#ff0000",
            "#a80000",
            "#6e0000",
            "#ffbee8",
            "#ff73df",
        ]
    )
    # MPH
    bounds = np.array(
        [0.0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 100, 120, 140, 160]
    )
    norm, norm2 = make_custom_cmaps(name, colors, bounds)
    cmap = plt.get_cmap(name)
    cmap2 = plt.get_cmap(name + "2")
    kwargs = dict(cmap=cmap, norm=norm)
    kwargs2 = dict(cmap=cmap, norm=norm2)
    cbar_kwargs = dict(label=f"{variable} ({units})")
    cbar_kwargs2 = cbar_kwargs | dict(spacing="proportional", ticks=bounds)

class NWSWindSpeedkts:
    name = "nws.wind"
    units = r"kts"
    variable = "Wind Speed"
    colors = np.array(
        [
            "#103f78",
            "#225ea8",
            "#1d91c0",
            "#41b6c4",
            "#7fcdbb",
            "#b4d79e",
            "#dfff9e",
            "#ffffa6",
            "#ffe873",
            "#ffc400",
            "#ffaa00",
            "#ff5900",
            "#ff0000",
            "#a80000",
            "#6e0000",
            "#ffbee8",
            "#ff73df",
        ]
    )
    # kts
    bounds = np.array(
        [0.0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 100, 120, 140, 160]
    )
    norm, norm2 = make_custom_cmaps(name, colors, bounds)
    cmap = plt.get_cmap(name)
    cmap2 = plt.get_cmap(name + "2")
    kwargs = dict(cmap=cmap, norm=norm)
    kwargs2 = dict(cmap=cmap, norm=norm2)
    cbar_kwargs = dict(label=f"{variable} ({units})")
    cbar_kwargs2 = cbar_kwargs | dict(spacing="proportional", ticks=bounds)
'''
# Get a list of all colormap names
colormaps = plt.colormaps()

# Print each colormap name
for cmap in colormaps:
    print(cmap)
'''
###########################################################################
##################### Config ##############################################

model = "rrfs" #@param ["hrrrak", "namnest", "rrfs"]

#@markdown Choose Model Run Date
rundate = "2024-08-07" #@param {type:"date"}
#@markdown Choose Model Run Hour (UTC)
hour = 0 #@param {type:"slider", min:0, max:21, step:3}

runhour = str(hour).zfill(2)

runstring = rundate + " " +runhour + ":00"

run_time = rundate + " " +runhour + "Z"

#@markdown Choose your forecast start hour
start_hour = 12 #@param {type:"slider", min:0, max:48, step:3}

#@markdown Choose your forecast end hour
end_hour = 24 #@param {type:"slider", min:0, max:48, step:3}

# Ensure end_hour is greater than start_hour
if end_hour < start_hour:
    end_hour = start_hour

#@markdown Would you like to create an animated loop of your images?
animated = True #@param {type:"boolean"}
#@markdown Choose your weather element to plot (current choices are: **6hrPrecip, 1hrPrecip, 2mTemp, Reflectivity, 10mWind, TotalPrecip, SBCAPE, Shear6km**)
#@markdown If you select TotalPrecip, you will get the accumulated precipitation between your end and start times. Note that both time steps have to exist in the archive for this to happen.
element = "10mWind" #@param ["6hrPrecip","2mTemp","Reflectivity", "10mWind", "TotalPrecip", "SBCAPE", "Shear6km"]

timestep = 3

# Generate the list of forecast hours from start to end with a step of 3
forecast_hours = list(range(start_hour, end_hour + 1, timestep))
print(f"Selected forecast hour range: {start_hour} to {end_hour}")
print(f"Forecast hours list: {forecast_hours}")

#@markdown If you're plotting wind (or wind shear), choose your thinning factor for wind barbs (50 is a good number for the full AK domain).  For small domains try a smaller number like 10 or 20.
thin_factor = 5 #@param {type:"integer"}

product = "sfc"

#@markdown What zoom would you like to use?  Default is full domain or you can check the custom zoom box below to create your own zoom level
zoom = "AncBowl" #@param ["full", "AncBowl", "SEAK", "SAK", "FairbanksArea", "Custom"]
#@markdown Would you like to have a custom zoom?  If so, make sure your domain is "Custom" and enter appropriate values for lat/lons below

custom_west = -135.00 #@param {type:"number"}
custom_north = 57.39 #@param {type:"number"}
custom_east = -130.46 #@param {type:"number"}
custom_south = 54.21 #@param {type:"number"}

if zoom == "Custom":
  # Check if the values are valid
  print("I see you've selected a custom zoom...checking your lat/lon pairs...")
  if custom_west is not None and custom_east is not None and custom_north is not None and custom_south is not None:
      if (-180 <= custom_west <= 180) and (-180 <= custom_east <= 180) and (-90 <= custom_north <= 90) and (-90 <= custom_south <= 90):
          print("All domain bounds have valid values.")
      else:
          print("One or more of your lat/lon pairs is outside the valid range.  Defaulting to full domain...")
          zoom = "full"
  else:
      print("One or more of your lat/lon values are missing.  Defaulting to full domain...")
      zoom = "full"

#@markdown Your files will save to /nas/hires/graphics.  To find them either click the link or open your
#@markdown file repo by clicking on the folder icon on the left toolbar.  Navigate
#@markdown to the folder with the two dots following ( ..) which will open up the main
#@markdown file system you get with colab.  Then follow the path above!

graphicsdir = '/nas/hires/graphics'


domain_dict = {
    "SEAK": {
        "west": -145.,
        "south": 53,
        "east": -129.,
        "north": 61.
    },
    "SAK": {
        "west": -160.,
        "south": 56.,
        "east": -140.,
        "north": 63.
    },
    "AncBowl": {
        "west": -153.,
        "south": 60.,
        "east": -144.,
        "north": 62.5
    },
    "FairbanksArea": {
        "west": -150.,
        "south": 64.,
        "east": -145.,
        "north": 65.6
    },
    "Custom": {
        "west": custom_west,
        "south": custom_south,
        "east": custom_east,
        "north": custom_north
    }
}

if zoom in domain_dict:
  west = domain_dict[zoom]["west"]
  south = domain_dict[zoom]["south"]
  east = domain_dict[zoom]["east"]
  north = domain_dict[zoom]["north"]
else:
  print(f"Did not understand your domain selection. Plotting for full domain...")
  zoom = "full"
################################# Main Code ################################
# this is for doing total precip/snow
tr_begin = forecast_hours[0]
tr_end = forecast_hours[-1]

# logic for namnest total precipitation since it doesn't seem to have true
# accumulated precip

accum_precip = []
if model == "namnest" and element == "TotalPrecip":
  for fcst_hr in forecast_hours:
    if fcst_hr != tr_begin:
      try:
        H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
        ds = H.xarray(":APCP:")
        precip = ds.tp/25.4
        accum_precip.append(precip)
      except Exception as e:
        print(e)
        continue

loop_graphics = []
for fcst_hr in forecast_hours:
  valid_time = (datetime.strptime(runstring, "%Y-%m-%d %H:%M") + timedelta(hours=fcst_hr)).strftime("%Y-%m-%d %HZ")
  # logic for selecting variables
  if element == "6hrPrecip":
    print(f"Now working on {model} {element} for valid time of {valid_time}")
    try:
    #have to subtract the previous total precip accum from the current
    #This won't work unless we have both time steps
    # getting the correct run based on inputs
      if model == "hrrrak":
        H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
        H2 = Herbie(runstring, model=model, product=product, fxx=fcst_hr-6)
        ds = H.xarray(":APCP:surface:0-[1-9]*")
        ds2 = H2.xarray(":APCP:surface:0-[1-9]*")
        late = ds.tp/25.4
        early = ds2.tp/25.4
        oldvar = late-early
      elif model == "namnest":
        H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
        H2 = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr-3)
        ds = H.xarray(":APCP:")
        ds2 = H2.xarray(":APCP:")
        now = ds.tp/25.4
        prev = ds2.tp/25.4
        oldvar = now+prev
      elif model == "rrfs":
        H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
        H2 = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr-6)
        ds = H.xarray(":APCP:surface:0-[1-9]*")
        ds2 = H2.xarray(":APCP:surface:0-[1-9]*")
        late = ds.tp/25.4
        early = ds2.tp/25.4
        oldvar = late-early

      var = oldvar.where(oldvar>0.009)
      name = f"6hr {ds.tp.GRIB_name.split(' ')[-1]}"
      description = ds.description
      kwargs = NWSPrecipitation.kwargs2
      cbar_kwargs = NWSPrecipitation.cbar_kwargs2
    except Exception as e:
      print(e)
      continue

  elif element == "TotalPrecip":
    # There's probably a more efficient way to do this but oh well...
    if fcst_hr == tr_end:
      print(f"Now working on {model} {element} for valid time of {valid_time}")
      try:
        #have to subtract the starting total precip accum from the current
        #This won't work unless we have both time steps
        # getting the correct run based on inputs
        if model == "hrrrak":
          H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
          H2 = Herbie(runstring, model=model, product=product, fxx=tr_begin)
          ds = H.xarray(":APCP:surface:0-[1-9]*")
          ds2 = H2.xarray(":APCP:surface:0-[1-9]*")
        elif model == "namnest":
          H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
          H2 = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=tr_begin)
          ds = H.xarray(":APCP:")
          ds2 = H2.xarray(":APCP:")
        elif model == "rrfs":
          if tr_begin == 0:
            H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
            ds = H.xarray(":APCP:surface:0-[1-9]*")
          else:
            H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
            H2 = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=tr_begin)
            ds = H.xarray(":APCP:surface:0-[1-9]*")
            ds2 = H2.xarray(":APCP:surface:0-[1-9]*")
        if model != "namnest":
          if tr_begin == 0:
            oldvar = ds.tp/25.4
          else:
            end = ds.tp/25.4
            begin = ds2.tp/25.4
            oldvar = end-begin
          var = oldvar.where(oldvar>0.009)
        else:
          # summing the list we created earlier
          oldvar = sum(accum_precip)
          var = oldvar.where(oldvar>0.009)
        name = f"Total {ds.tp.GRIB_name.split(' ')[-1]} from F{tr_begin:02d} to F{tr_end:02d}"
        description = ds.description
        kwargs = NWSPrecipitation.kwargs2
        cbar_kwargs = NWSPrecipitation.cbar_kwargs2
      except Exception as e:
        print(e)
        continue
    else:
      continue

  elif element == "2mTemp":
    print(f"Now working on {model} {element} for valid time of {valid_time}")
    # getting the correct run based on inputs
    try:
      if model == "hrrrak":
        H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
      elif model == "namnest":
        H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
      elif model == "rrfs":
        H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
    except Exception as e:
      print(e)
      continue
    ds = H.xarray(":TMP:2 m above")
    var = ds.t2m-273.15
    name = ds.t2m.GRIB_name
    description = ds.description
    kwargs = paint.NWSTemperature.kwargs2
    cbar_kwargs = paint.NWSTemperature.cbar_kwargs2

  elif element == "SBCAPE":
    print(f"Now working on {model} {element} for valid time of {valid_time}")
    # getting the correct run based on inputs
    try:
      if model == "hrrrak":
        H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
      elif model == "namnest":
        H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
      elif model == "rrfs":
        H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
    except Exception as e:
      print(e)
      continue
    ds = H.xarray(":CAPE:surface")
    #print(ds)
    var = ds.cape.where(ds.cape>10)
    name = ds.cape.GRIB_name
    description = ds.description
    #kwargs = paint.NWSTemperature.kwargs2
    #cbar_kwargs = paint.NWSTemperature.cbar_kwargs2

  elif element == "Reflectivity":
    print(f"Now working on {model} {element} for valid time of {valid_time}")
    # getting the correct run based on inputs
    try:
      if model == "hrrrak":
        H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
        ds = H.xarray(":REFC:")
        var = ds.refc.where(ds.refc>0)
        name = ds.refc.GRIB_name
      elif model == "namnest":
        H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
        ds = H.xarray(":REFC:")
        var = ds.refc.where(ds.refc>0)
        name = ds.refc.GRIB_name
      elif model == "rrfs":
        H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
        ds = H.xarray(":REFC:")
        var = ds.unknown.where(ds.unknown>0)
        name = ds.unknown.GRIB_name
    except Exception as e:
      print(e)
      continue

    description = ds.description
    #kwargs = paint.NWSRefectivity.kwargs2
    #cbar_kwargs = paint.NWSReflectivity.cbar_kwargs2

  elif element == "10mWind":
    print(f"Now working on {model} {element} for valid time of {valid_time}")
    try:
      if model == "hrrrak":
        H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
        H2 = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
        ds = H.xarray(":UGRD:10 m above")
        ds2 = H2.xarray(":VGRD:10 m above")
        # creating speed and direction products
        mag = np.sqrt(ds.u10**2 + ds2.v10**2)
        # Thin out the wind barb data by slicing
        u_thin = ds.u10[::thin_factor, ::thin_factor]*2.23694
        v_thin = ds2.v10[::thin_factor, ::thin_factor]*2.23694
        lon_thin = ds.longitude[::thin_factor, ::thin_factor]
        lat_thin = ds.latitude[::thin_factor, ::thin_factor]

      elif model == "namnest":
        # Nam weirdness requires just subsetting "VGRD" and of course you get both U and V??
        H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
        ds = H2.xarray(":VGRD:10 m above")
        # creating speed and direction products
        mag = np.sqrt(ds.u10**2 + ds.v10**2)
        # Thin out the wind barb data by slicing
        u_thin = ds.u10[::thin_factor, ::thin_factor]*2.23694
        v_thin = ds.v10[::thin_factor, ::thin_factor]*2.23694
        lon_thin = ds.longitude[::thin_factor, ::thin_factor]
        lat_thin = ds.latitude[::thin_factor, ::thin_factor]

      elif model == "rrfs":
        H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
        H2 = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
        ds = H.xarray(":UGRD:10 m above")
        ds2 = H2.xarray(":VGRD:10 m above")
        # creating speed and direction products
        mag = np.sqrt(ds.u10**2 + ds2.v10**2)
        # Thin out the wind barb data by slicing
        u_thin = ds.u10[::thin_factor, ::thin_factor]*2.23694
        v_thin = ds2.v10[::thin_factor, ::thin_factor]*2.23694
        lon_thin = ds.longitude[::thin_factor, ::thin_factor]
        lat_thin = ds.latitude[::thin_factor, ::thin_factor]
    except Exception as e:
      print(e)
      continue

    name = "10m Wind"
    description = ds.description
    kwargs = NWSWindSpeed.kwargs2
    cbar_kwargs = NWSWindSpeed.cbar_kwargs2
    var = mag*2.23694 #converting to mph

  elif element == "Shear6km":
    print(f"Now working on {model} {element} for valid time of {valid_time}")
    try:
      if model == "hrrrak":
        H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
        H2 = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
      elif model == "namnest":
        H = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
        H2 = Herbie(runstring, model='nam', product='alaskanest.hiresf', fxx=fcst_hr)
      elif model == "rrfs":
        H = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
        H2 = Herbie(runstring, model="rrfs", member='control', domain="alaska", fxx=fcst_hr)
    except Exception as e:
      print(e)
      continue
    ds = H.xarray(":VUCSH:0-6000 m")
    ds2 = H2.xarray(":VVCSH:0-6000 m")
    # creating speed and direction products
    #print(ds)
    mag = np.sqrt(ds.vucsh**2 + ds2.vvcsh**2)
    name = "0-6km Shear"
    description = ds.description
    kwargs = NWSWindSpeedkts.kwargs2
    cbar_kwargs = NWSWindSpeedkts.cbar_kwargs2
    var = mag*1.94384 #converting to kts
    # Thin out the wind barb data by slicing
    u_thin = ds.vucsh[::thin_factor, ::thin_factor]*1.94384
    v_thin = ds2.vvcsh[::thin_factor, ::thin_factor]*1.94384
    lon_thin = ds.longitude[::thin_factor, ::thin_factor]
    lat_thin = ds.latitude[::thin_factor, ::thin_factor]

  modelname = ds.model.upper()

# setting up our plots
  ax = EasyMap("50m", crs=ds.herbie.crs, figsize=[10,8]).STATES().OCEAN().LAND().ax

  # setting custom extent if necessary
  if zoom != "full":
    ax.set_extent([west, east, south, north])
  if element == "2mTemp" or element == "6hrPrecip" or element == "10mWind" or element == "TotalPrecip" or element == "Shear6km":
    #plotting magnitude fields
    p = ax.pcolormesh(ds.longitude, ds.latitude, var, transform=pc, **kwargs)
    # overlaying wind barbs if wind is selected
    if element == "10mWind" or element == "Shear6km":
      ax.barbs(lon_thin.values, lat_thin.values, u_thin.values, v_thin.values, length=6, transform=pc)

    plt.colorbar(p, ax=ax, orientation="horizontal", pad=0.05, **cbar_kwargs)
    if model == 'namnest':
      ax.set_title(f"NAM:AlaskaNest--Run: {run_time}\n{name} Valid {valid_time}", loc="center", pad=5)
    elif model == 'rrfs':
      ax.set_title(f"{modelname}--Control Run: {run_time}\n{name} Valid {valid_time}", loc="center", pad=5)
    else:
      ax.set_title(f"{modelname}--Run: {run_time}\n{name} Valid {valid_time}", loc="center", pad=5)

  elif element == "Reflectivity":
    p = ax.pcolormesh(ds.longitude, ds.latitude, var, transform=pc, cmap='radar.reflectivity', vmin=0, vmax=70)

    plt.colorbar(p, ax=ax, orientation="horizontal", pad=0.05)

    if model == 'namnest':
      ax.set_title(f"NAM:AlaskaNest--Run: {run_time}\n{name} Valid {valid_time}", loc="center", pad=5)
    elif model == 'rrfs':
      ax.set_title(f"{modelname}--Control Run: {run_time}\n{name} Valid {valid_time}", loc="center", pad=5)
    else:
      ax.set_title(f"{modelname}--Run: {run_time}\n{name} Valid {valid_time}", loc="center", pad=5)

  elif element == "SBCAPE":
    p = ax.pcolormesh(ds.longitude, ds.latitude, var, transform=pc, cmap='hot_r', vmin=0, vmax=3000)

    plt.colorbar(p, ax=ax, orientation="horizontal", pad=0.05)

    if model == 'namnest':
      ax.set_title(f"NAM:AlaskaNest--Run: {run_time}\n{name} Valid {valid_time}", loc="center", pad=5)
    elif model == 'rrfs':
      ax.set_title(f"{modelname}--Control Run: {run_time}\n{name} Valid {valid_time}", loc="center", pad=5)
    else:
      ax.set_title(f"{modelname}--Run: {run_time}\n{name} Valid {valid_time}", loc="center", pad=5)

  # setting the title for our graphic
  graphictitle = f'Surface {modelname}_{element}_{run_time.replace("-","").replace(" ","")}_{valid_time.replace("-","").replace(" ","")}.png'

  if not os.path.exists(graphicsdir):
    os.makedirs(graphicsdir)
  # appending graphic title to list of images for loop
  loop_graphics.append(graphictitle)
  plt.savefig(f"{graphicsdir}/{graphictitle}")
  plt.show()
  print(f"Saved {graphictitle} to {graphicsdir}")
  plt.close()

# creating animation if requested
if animated:
  print("Creating animation...")
  # check for length of loop
  if len(loop_graphics) <= 1:
    print("You don't have enough images for a loop...sorry!")
  else:
    # Open images and store them in a list
    frames = [Image.open(os.path.join(graphicsdir,plot)) for plot in loop_graphics]
    # Create a list of durations (500 ms for all frames except the last one, which will be 1000 ms)
    durations = [500] * (len(frames) - 1) + [1000]  # Last frame will be shown for 1000 ms (twice as long)

    # Save the frames as an animated GIF with custom durations
    frames[0].save(os.path.join(graphicsdir,f"{model}_{element}_animation.gif"), format="GIF", append_images=frames[1:], save_all=True, duration=durations, loop=0)

    print(f"Animation saved as {model}_{element}_animation.gif")

